In [5]:
import pandas as pd
data_url = "./country_hsproduct2digit_year.csv.zip"
data = pd.read_csv(data_url, compression="zip", low_memory=False)
data = data[['year','location_code','hs_product_code','export_value']]

In [6]:
from ecomplexity import ecomplexity
from ecomplexity import proximity

# Calculate complexity
trade_cols = {'time':'year', 'loc':'location_code', 'prod':'hs_product_code', 'val':'export_value'}
cdata = ecomplexity(data, trade_cols)

# Calculate proximity matrix
prox_df = proximity(data, trade_cols)

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [9]:
prox_df 

,year,hs_product_code_1,hs_product_code_2,proximity
1,1995,01,02,0.354167
2,1995,01,03,0.207921
3,1995,01,04,0.354167
4,1995,01,05,0.292308
5,1995,01,06,0.145833
6,1995,01,07,0.362319
7,1995,01,08,0.237500
8,1995,01,09,0.151515
9,1995,01,10,0.312500
10,1995,01,11,0.437500


In [13]:
data.keys()

Index(['year', 'location_code', 'hs_product_code', 'export_value'], dtype='object')

In [1]:
from ecomplexity import ecomplexity

ModuleNotFoundError: No module named 'ecomplexity'

In [2]:
pip show ecomplexity



The following command must be run outside of the IPython shell:

    $ pip show ecomplexity

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
pip show complexity


The following command must be run outside of the IPython shell:

    $ pip show complexity

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/
